## 研究の流れ

このipynb と README.md を見るだけで流れがわかるようにしました。  
なお、各処理の詳細が知りたい場合は各ファイルを閲覧してください。


In [ ]:
# 標準モジュールのインポート
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import subprocess
from glob import glob

# src配下のファイルを読み込めるようにする
sys.path.append(os.getcwd() + "/src")


In [ ]:
# 共通する自作モジュールのインポート
from Visualization.SnapData import SnapData
from SetLogger import logger_conf
# logger = logger_conf()


In [ ]:
# ディレクトリの作成と定義
from params import ROOT_DIR, SNAP_PATH, SRC_PATH, ML_DATA_DIR, ML_RESULT_DIR, datasets, variable_parameters
data_dir = ROOT_DIR + "/data"

# 出力先フォルダの作成
subprocess.run([ROOT_DIR+"/etc/BasePath.bat"])


### 1. データの加工

##### 1.1 元データを各種パラメータに分割する

In [ ]:
# 標準モジュールのインポート
import os
import sys

# src配下のファイルを読み込めるようにする
sys.path.append(os.getcwd() + "/src")

from Processing.separater_loop import data_processing


In [ ]:
data_processing()


##### 1.2 バイナリを .npy に変換

In [ ]:
import os
import sys
from glob import glob
sys.path.append(os.getcwd() + "/src")

from params import SNAP_PATH, datasets, variable_parameters
from Processing.snap2npy import snap2npy


In [ ]:
sp = SnapData()

for dataset in datasets:
    for val_param in variable_parameters:
        for path in glob(SNAP_PATH + f"/snap{dataset}/{val_param}/*/*"):
            # print(path)
            snap2npy(sp, path, dataset)


## 2. 可視化

#### 2.1 各種可視化

In [ ]:
# 標準モジュールのインポート
import os
import sys
from glob import glob

# src配下のファイルを読み込めるようにする
sys.path.append(os.getcwd() + "/src")


from params import SNAP_PATH, datasets
from Visualization.Visualize import VisualizeMethod


In [ ]:
for dataset in datasets:
    target_path = SNAP_PATH + f"/snap{dataset}"
    viz = VisualizeMethod(dataset)

    files = {}
    files["density"] = glob(target_path + f"/density/*/*")
    files["velocityx"] = glob(target_path + f"/velocityx/*/*")
    files["velocityy"] = glob(target_path + f"/velocityy/*/*")
    for dens_path, vx_path, vy_path in zip(files["density"], files["velocityx"], files["velocityy"]):
        viz.drawEnergy_for_velocity(dens_path, vx_path, vy_path)

    files["magfieldx"] = glob(target_path + f"/magfieldx/*/*")
    files["magfieldy"] = glob(target_path + f"/magfieldy/*/*")
    for magx_path, magy_path in zip(files["magfieldx"], files["magfieldy"]):
        viz.drawEnergy_for_magfield(magx_path, magy_path)
        
    files["enstrophy"] = glob(target_path + f"/enstrophy/*/*")
    for target in ["velocityx", "velocityy", "magfieldx", "magfieldy", "density", "enstrophy"]:
        for path in files[target]:
            viz.drawHeatmap(path)
            viz.drawEdge(path)


#### 2.2 AVS

#### 2.3 StreamLines

#### 2.4 LIC

In [ ]:
import os
import sys
from glob import glob
sys.path.append(os.getcwd() + "/src")

from params import SNAP_PATH, IMGOUT, datasets
from LIC.LIC import LicMethod



In [ ]:
def mainProcess(logger, lic: LicMethod, dir_basename: str, base_out_path: str, binary_paths: list[str]):
    for xfile in binary_paths:
        try:
            logger.debug("START", extra={"addinfo": f"{os.path.splitext(os.path.basename(xfile))[0]} 開始\n"})
            yfile = xfile.replace("magfieldx", "magfieldy")
            file_name = os.path.splitext(os.path.basename(xfile.replace("magfieldx", "magfield")))
            out_path = base_out_path + f"/lic_{dir_basename}.{file_name[0]}.bmp"
            print(out_path) # ./imgout/LIC/snap77/lic_snap77.magfieldx.01.14.bmp

            if not os.path.exists(out_path):
                # 引数の作成
                command = lic.set_command(xfile, yfile, out_path)
                # 実行 (1画像30分程度)
                # 22:12:46 -> 22:46:12 (34分)
                lic.LIC(command)

                # temp ファイルの削除
                lic.delete_tempfile(command[1], command[2])
            logger.debug("END", extra={"addinfo": f"{os.path.splitext(os.path.basename(xfile))[0]} 終了\n"})

        except KeyboardInterrupt:
            break

        except Exception as e:
            logger.debug(str(e), extra={"addinfo": f"{os.path.splitext(os.path.basename(xfile))[0]} 中断\n"})



In [ ]:
"""
処理時間の目安
snap77   : 778(ファイル) * 30(分) / 60 / 4 (並列スレッド数) * (CPU速度(GHz) / 2.8(GHz))
-> 64.833 (時間)

snap497  : 791(ファイル) * 30(分) / 60 / 4 (並列スレッド数) * (CPU速度(GHz) / 2.8(GHz))
-> 65.9167 (時間)

snap4949 : 886(ファイル) * 30(分) / 60 / 4 (並列スレッド数) * (CPU速度(GHz) / 2.8(GHz))
-> 73.83 (時間)

計     : 2455 * 30(分) / 60 / 並列スレッド数 * (CPU速度(GHz) / 2.8(GHz))
-> 204.58 (時間)

"""

from SetLogger import logger_conf
from params import datasets
from concurrent.futures import ThreadPoolExecutor

try:
    # ログ取得の開始
    logger = logger_conf()
    logger.debug("START", extra={"addinfo": "処理開始\n\n"})

    dataset = int(input("使用するデータセットを入力してください (77/497/4949): "))
    if dataset not in datasets:
        logger.debug("ERROR", extra={"addinfo": "このデータセットは使用できません\n"})
        sys.exit()

    logger.debug("START", extra={"addinfo": f"{dataset} 開始\n"})
    lic = LicMethod()
    
    # 入出力用path の作成
    indir = SNAP_PATH + f"/half/snap{dataset}"
    dir_basename = os.path.basename(indir) # snap77
    out_dir = IMGOUT + "/LIC"
    base_out_path = out_dir + "/" + os.path.basename(indir) # ./imgout/LIC/snap77
    lic.makedir(f"/LIC/snap{dataset}")

    # バイナリファイルの取得
    binary_paths = glob(indir+"/magfieldx/*/*.npy")

    # ファイルが無い場合
    if binary_paths == []:
        logger.debug("ERROR", extra={"addinfo": f"File not Found\n"})
        sys.exit()

    file_count = len(binary_paths)
    with ThreadPoolExecutor(max_workers=6) as exec: # 並列処理 # max_workers は自信のCPUのコア数と相談してください
        exec.submit(mainProcess, logger, lic, dir_basename, base_out_path, binary_paths[: file_count // 6])
        exec.submit(mainProcess, logger, lic, dir_basename, base_out_path, binary_paths[file_count // 6 : file_count // 6 * 2])
        exec.submit(mainProcess, logger, lic, dir_basename, base_out_path, binary_paths[file_count // 6 * 2 : file_count // 6 * 3])
        exec.submit(mainProcess, logger, lic, dir_basename, base_out_path, binary_paths[file_count // 6 * 3 : file_count // 6 * 4])
        exec.submit(mainProcess, logger, lic, dir_basename, base_out_path, binary_paths[file_count // 6 * 4 : file_count // 6 * 5])
        exec.submit(mainProcess, logger, lic, dir_basename, base_out_path, binary_paths[file_count // 6 * 5 :])
    
    logger.debug("END", extra={"addinfo": f"{dataset} 終了\n"})

except KeyboardInterrupt as e:
    logger.debug("ERROR", extra={"addinfo": f"処理中断\n"})

except Exception as e:
    logger.debug("ERROR", extra={"addinfo": f"{e}\n"})

finally:
    logger.debug("END", extra={"addinfo": "処理終了"})



### 3. 教師データの作成

#### 3-1 ビューワの作成

In [1]:
import os
import sys
sys.path.append(os.getcwd() + "/src")

from Processing.viewer.createViewer import createViewer

In [2]:
dataset = input("使用するデータセットを入力してください(77/497/4949) : ")
if dataset not in datasets:
    sys.exit()

createViewer(dataset)


#### 3-2 画像の分割

`python ./src/Processing/viewer/writer.py`  
を実行し、  
`./src/Processing/viewer/template/lic_viewer77.html`
をWebで開く (Drug & Drop)


### 3.3 

### 4. 機械学習

#### 4.1 SVM

#### 4.2 非線形SVM

#### 4.3 k-Means

In [ ]:
import os
import sys
import numpy as np
from glob import glob
sys.path.append(os.getcwd() + "\src")

from params import SNAP_PATH, datasets, variable_parameters
from k_means.Clustering import ClusteringMethod


In [ ]:
cluster = ClusteringMethod()

for dataset in datasets:
    for target in variable_parameters:
        path_list = glob(SNAP_PATH + f"/snap{dataset}/{target}/*/*")
        num_of_data = len(path_list) # リコネクションがない画像の枚数

        temp_data = cluster.compress(cluster.loadSnapData(path_list[0],z=3))
        IMGSHAPE = temp_data.shape

        N_col = IMGSHAPE[0] * IMGSHAPE[1] * 1 # 行列の列数
        X_train = np.zeros((num_of_data, N_col)) # 学習データ格納のためゼロ行列生成
        y_train = np.zeros((num_of_data)) # 学習データに対するラベルを格納するためのゼロ行列生成

        # リコネクションがない画像を行列に読み込む
        for idx, item in enumerate(path_list[:10]):
            X_train[idx, :] = cluster.load_regularize(item)
            y_train[idx] = 0 # リコネクションがないことを表すラベル

        X_train_pca = cluster.PCA(X_train)
        cluster_labels = cluster.KMeans(X_train_pca)
        df_re = cluster.save_result(cluster_labels, path_list, dataset)
        # display(df_re)


#### 4.4 XGBoost